In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import imblearn
import collections

import sklearn
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report,confusion_matrix

from keras.utils import to_categorical

from keras.utils.vis_utils import plot_model

from collections import Counter
from imblearn.under_sampling import OneSidedSelection
from imblearn.over_sampling import ADASYN

import pickle

Load the Data

In [2]:
csv_file = 'actionstatespaceTLorenzoUStella_combined_final.csv'
dataframe = pd.read_csv(csv_file)

In [3]:
dataframe.head()

#print(dataframe["Time Since Last Charge"].mean())
#print(dataframe["Time Since Last Action Formatted"].mean())

,Action,Last Action,Session Time,Time Since Last Charge,Session Time Formatted,Time Since Last Action,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,...,st1,st2,st3,st4,st5,st6,st7,st8,st9,st10
0,2,17,0:01:44.978445,0.00,1.73,0:00:00.000000,0.00,Happy,1,None,...,0,0,01:44.978445,01:44,1,1,1,44,0.73,1.73
1,3,2,0:02:02.620733,0.30,2.03,0:00:17.642288,0.28,Happy,1,None,...,0,0,02:02.620733,02:02,2,2,2,2,0.03,2.03
2,3,3,0:02:16.291131,0.54,2.27,0:00:13.670398,0.22,Happy,1,Low,...,0,0,02:16.291131,02:16,2,2,2,16,0.27,2.27
3,7,3,0:02:29.106355,0.75,2.48,0:00:12.815224,0.20,Happy,1,Low,...,0,0,02:29.106355,02:29,2,2,2,29,0.48,2.48
4,9,7,0:02:54.246610,1.17,2.90,0:00:25.140255,0.42,Sad,1,Low,...,0,0,02:54.246610,02:54,2,2,2,54,0.90,2.90


Format and One-hot encode the data

In [4]:
dataframe.drop("tsla1", axis=1, inplace=True)
dataframe.drop("tsla2", axis=1, inplace=True)
dataframe.drop("tsla3", axis=1, inplace=True)
dataframe.drop("tsla4", axis=1, inplace=True)
dataframe.drop("tsla5", axis=1, inplace=True)
dataframe.drop("tsla6", axis=1, inplace=True)
dataframe.drop("st1", axis=1, inplace=True)
dataframe.drop("st2", axis=1, inplace=True)
dataframe.drop("st3", axis=1, inplace=True)
dataframe.drop("st4", axis=1, inplace=True)
dataframe.drop("st5", axis=1, inplace=True)
dataframe.drop("st6", axis=1, inplace=True)
dataframe.drop("st7", axis=1, inplace=True)
dataframe.drop("st8", axis=1, inplace=True)
dataframe.drop("st9", axis=1, inplace=True)
dataframe.drop("st10", axis=1, inplace=True)
dataframe.drop("Session Time", axis=1, inplace=True)
dataframe.drop("Time Since Last Action", axis=1, inplace=True)

dataframe.drop("Session Time Formatted", axis=1, inplace=True)
#dataframe.drop("Time Since Last Charge", axis=1, inplace=True)  

In [5]:
dataframe.head(100)

,Action,Last Action,Time Since Last Charge,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,Input Mood,Activity
0,2,17,0.00,0.00,Happy,1,None,Neutral,Studying
1,3,2,0.30,0.28,Happy,1,None,Neutral,Studying
2,3,3,0.54,0.22,Happy,1,Low,Neutral,Studying
3,7,3,0.75,0.20,Happy,1,Low,Neutral,Studying
4,9,7,1.17,0.42,Sad,1,Low,Neutral,Studying
...,...,...,...,...,...,...,...,...,...
95,12,12,35.98,1.62,Sad,1,None,Neutral,Studying
96,11,12,37.38,1.38,Sad,1,None,Positive,Relaxing
97,15,11,40.17,2.78,Neutral,0,Low,Positive,Relaxing
98,15,15,40.93,0.77,Sad,1,None,Positive,Relaxing


In [6]:
pd.set_option('display.max_columns', None)
dataframe = pd.get_dummies(dataframe, columns=["Activity"], prefix=["Activity"])
dataframe = pd.get_dummies(dataframe, columns=["Camera Mood"], prefix=["Camera_Mood"])
dataframe = pd.get_dummies(dataframe, columns=["Input Mood"], prefix=["Input Mood"])
#both = pd.concat([dataframe, dum_df])
#dataframe.drop("Input Mood", axis=1, inplace=True) 
#both.head(5)

dataframe = pd.get_dummies(dataframe, columns=["Noise"], prefix=["Noise"])
dataframe = pd.get_dummies(dataframe, columns=["Last Action"], prefix=["Last Action"])
dataframe.head(100)

,Action,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Fear,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,2,0.00,0.00,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,3,0.30,0.28,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0.54,0.22,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7,0.75,0.20,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,1.17,0.42,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,12,35.98,1.62,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
96,11,37.38,1.38,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
97,15,40.17,2.78,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
98,15,40.93,0.77,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [7]:
X = dataframe.drop('Action', axis=1)
y = dataframe['Action']

In [8]:
X.head(5)

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Fear,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,0.00,0.00,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.30,0.28,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.54,0.22,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.75,0.20,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.17,0.42,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [9]:
X_train = X[0:148]
X_test = X[148:209]
y_train = y[0:148]
y_test = y[148:209]

In [10]:
X_train

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Fear,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,0.00,0.00,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.30,0.28,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.54,0.22,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.75,0.20,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.17,0.42,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,38.58,0.37,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
144,0.00,14.98,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
145,0.42,0.40,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
146,1.52,1.10,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [11]:
counter = Counter(y_train)
print(counter)

Counter({16: 20, 7: 19, 9: 15, 10: 13, 15: 13, 12: 12, 3: 8, 14: 8, 11: 7, 1: 6, 6: 5, 13: 5, 2: 4, 4: 4, 17: 4, 8: 3, 5: 2})


In [12]:
counter = Counter(y_test)
print(counter)

Counter({8: 10, 1: 7, 12: 6, 7: 5, 15: 5, 10: 4, 9: 4, 16: 4, 11: 3, 4: 3, 3: 2, 14: 2, 6: 2, 5: 2, 17: 1, 13: 1})


Normalize data (train and test done seperately)

In [13]:
X_train_scaled = X_train.copy()

col_names = ['Time Since Last Charge', 'Time Since Last Action Formatted']
features = X_train_scaled[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X_train_scaled[col_names] = features

filename = 'TLorenzoUStella_scaler2.sav'
#pickle.dump(scaler, open(filename, 'wb'))

In [14]:
scaler2 = pickle.load(open(filename, 'rb'))

X_test_scaled = X_test.copy()

features = X_test_scaled[col_names]
features = scaler2.transform(features.values)
X_test_scaled[col_names] = features

X_test_scaled.head()

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Fear,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Camera_Mood_Surprise,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
148,-0.996044,-0.595166,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
149,-0.962308,-0.355183,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
150,-0.736651,1.015390,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
151,-0.720158,-0.488507,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
152,-0.686422,-0.365849,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


Load and Train the Model

In [15]:
mlp = MLPClassifier(max_iter=10000, activation='relu', hidden_layer_sizes=(15)) #doesn't converge at 500 so put to higher, do we want it to converge?
mlp

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [16]:
mlp.fit(X_train_scaled,y_train)

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [35]:
#filename = 'TLorenzoUStella_model2.sav'
#pickle.dump(mlp, open(filename, 'wb'))

In [17]:
pred = mlp.predict(X_test_scaled)
pred

array([ 1, 13,  9, 14, 10, 10,  7, 14, 10, 15,  9,  9, 15, 15,  7,  4,  9,
       17,  3, 17, 17,  7,  7,  2, 10, 16, 10,  6, 11,  7,  4, 10,  7,  6,
        3,  4,  3,  6,  7,  7,  7, 16, 15, 15, 11,  7,  6, 15, 16, 15,  6,
        7, 16,  7,  7,  7,  7,  7,  7, 15,  7])

In [18]:
y_test.value_counts()

8     10
1      7
12     6
7      5
15     5
9      4
10     4
16     4
4      3
11     3
3      2
5      2
6      2
14     2
13     1
17     1
Name: Action, dtype: int64

In [19]:
from collections import Counter
Counter(pred)


Counter({1: 1,
         13: 1,
         9: 4,
         14: 2,
         10: 6,
         7: 18,
         15: 8,
         4: 3,
         17: 3,
         3: 3,
         2: 1,
         16: 4,
         6: 5,
         11: 2})

In [107]:
prediction = pd.DataFrame(pred, columns=['predictions']).to_csv('prediction.csv')